<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9b6da74a1cf2ef82610800b9f3e0e723508d2893d81515ce028dbfaa770a37fb
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-22 15:17:58
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 1.12 L (0.8%)
Current PnL: -25.70 L (-16.6%)
CY Booked + Current PnL: -11.41 L (-7.37%)
-------------------
Total profit:  1.69 L
Total loss:  -27.39 L
-------------------
Total Booked + Current PnL: 15.57 L (12.18%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.11%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.34 L (63.91%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.56%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,2.16,-17.90,21.91,0.09,18128.0,-18037.0,82740.0,92.10,44.0,M-SC,2.31,253.0,-0.99,0.59,8.70,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,1.08,8.76,10.05,19.69,20959.0,16803.0,208545.0,-1.91,55.0,X-MC,2.47,78.0,0.80,1.48,16.16,XY25,NTT,AC
50,MASFIN,398.61,0.08,-3.18,26.06,22.05,24721.0,-3120.0,94860.0,-16.44,51.0,H-SC,6.50,164.0,-0.13,0.67,37.89,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.61,-16.08,19.18,0.02,28098.0,-28065.0,146494.0,123.21,42.0,M-SC,12.61,234.0,-1.00,1.04,27.42,OX40N,NTT,PAINTS
52,NESTLEIND,1377.00,1.00,10.80,9.56,21.40,28115.0,28665.0,294091.0,2.77,56.0,X-LC,5.45,12.0,1.02,2.09,18.16,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,13.09,-55.35,124.13,0.07,76026.0,-75930.0,61247.0,2144.95,79.0,L-SC,3.14,268.0,-1.00,0.44,52.32,OX40N,NTT,JEWELLERY
74,TITAGARH,1548.0,3.89,-23.22,89.82,45.74,167006.0,-56243.0,185934.0,2.26,52.0,H-SC,6.36,174.0,-0.34,1.32,20.93,XY24,NTT,ENGINEERING
28,GREENPANEL,537.0,3.74,-37.18,127.86,43.14,147048.0,-68071.0,115007.0,157.75,40.0,M-SC,10.06,240.0,-0.46,0.82,8.18,XY24,NTT,MISC
13,BSOFT,831.7,3.71,-18.50,85.01,50.79,97451.0,-26018.0,114635.0,4.37,72.0,H-SC,7.25,151.0,-0.27,0.81,33.47,XR,ATH,IT
83,WHIRLPOOL,2270.0,3.34,-23.30,137.17,81.92,120782.0,-26745.0,88053.0,-55.05,35.0,M-SC,19.63,236.0,-0.22,0.63,5.04,XR,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-3.26,-14.53,47.50,26.07,79256.0,-28366.0,166855.0,-58.46,33.0,X-MC,11.39,56.0,-0.36,1.19,3.92,X40N,ATH,IT
18,COFFEEDAY,80.00,-2.64,-41.38,130.75,35.27,87035.0,-46983.0,66566.0,-55.73,47.0,L-SC,10.06,270.0,-0.54,0.47,62.92,XR,NTT,HOTELS
44,JCHAC,2282.00,-2.49,-36.68,57.90,-0.01,46859.0,-46874.0,80931.0,16465.22,33.0,M-SC,10.49,233.0,-1.00,0.58,3.58,OX40N,NTT,AC
8,AWL,485.00,-1.29,-23.51,100.66,53.50,232347.0,-70927.0,230824.0,-62.99,35.0,X-MC,4.80,58.0,-0.31,1.64,2.52,XY24,NTT,FMCG
57,RAJOOENG,143.10,-1.15,-37.21,122.34,39.61,78738.0,-38140.0,64360.0,-57.57,33.0,H-SC,30.04,135.0,-0.48,0.46,1.40,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.4,-0.69,109.35,107.91,160337.0,-1017.0,146627.0,-19.34,59.0,M-SC,12.05,216.0,-0.01,1.04,9.33,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.40,-0.69,109.35,107.91,160337.0,-1017.0,146627.0,-19.34,59.0,M-SC,12.05,216.0,-0.01,1.04,9.33,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.61,-16.08,19.18,0.02,28098.0,-28065.0,146494.0,123.21,42.0,M-SC,12.61,234.0,-1.00,1.04,27.42,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,2.16,-17.90,21.91,0.09,18128.0,-18037.0,82740.0,92.10,44.0,M-SC,2.31,253.0,-0.99,0.59,8.70,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,0.87,-21.30,67.77,32.03,70311.0,-28085.0,103750.0,-14.86,64.0,H-SC,9.33,148.0,-0.40,0.74,16.21,OX40N,NTT,IT
66,SIS,528.00,0.18,-23.41,58.32,21.26,49596.0,-25990.0,85042.0,2007.25,54.0,H-SC,4.71,166.0,-0.52,0.60,14.90,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,3.14,11.87,53.98,72.27,102329.0,20120.0,189568.0,-3.81,77.0,M-LC,3.38,99.0,0.2,1.35,18.78,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,3.14,11.87,53.98,72.27,102329.0,20120.0,189568.0,-3.81,77.0,M-LC,3.38,99.0,0.20,1.35,18.78,XR,NTT,IT
25,FINCABLES,1641.55,-0.40,-0.69,109.35,107.91,160337.0,-1017.0,146627.0,-19.34,59.0,M-SC,12.05,216.0,-0.01,1.04,9.33,OX40N,ATH,CABLES
6,ASIANTILES,137.00,2.68,-1.90,85.19,81.67,78401.0,-1779.0,92031.0,8288.89,74.0,L-SC,11.62,271.0,-0.02,0.65,78.22,XR,NTT,CERAMICS
38,INDIAMART,4810.62,0.06,-4.69,116.89,106.72,137409.0,-5782.0,117554.0,-52.73,37.0,H-SC,3.95,139.0,-0.04,0.84,17.59,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.10,-3.12,40.00,35.63,81213.0,-6541.0,203033.0,-16.74,40.0,H-MC,2.52,119.0,-0.08,1.44,13.03,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.62,-17.85,89.26,55.47,178391.0,-43430.0,199856.0,-27.18,32.0,X-MC,14.80,64.0,-0.24,1.42,2.40,X40N,ATH,FINANCE
22,DIXON,18931.72,-3.26,-14.53,47.50,26.07,79256.0,-28366.0,166855.0,-58.46,33.0,X-MC,11.39,56.0,-0.36,1.19,3.92,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.31,-38.87,121.17,35.20,95311.0,-50011.0,78659.0,3.10,34.0,X-SC,16.76,92.0,-0.52,0.56,0.89,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-0.62,-17.54,73.81,43.32,99127.0,-28572.0,134300.0,-29.17,34.0,X-SC,5.44,90.0,-0.29,0.95,3.12,X40N,ATH,ELECTRICAL
8,AWL,485.00,-1.29,-23.51,100.66,53.50,232347.0,-70927.0,230824.0,-62.99,35.0,X-MC,4.80,58.0,-0.31,1.64,2.52,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.42,-0.25,24.79,24.48,62280.0,-627.0,251229.0,-80.87,49.0,X-SC,0.94,86.0,-0.01,1.79,22.83,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.77,-3.42,25.43,21.14,55837.0,-7768.0,219571.0,-10.26,44.0,X-MC,1.14,74.0,-0.14,1.56,23.76,XY24,NTT,PAINTS
20,DABUR,735.0,-0.07,-3.50,48.91,43.70,117089.0,-8672.0,239396.0,-11.44,36.0,X-MC,2.05,72.0,-0.07,1.70,11.03,XY24,BTT,FMCG
43,ITC,452.0,0.42,-1.29,12.23,10.78,29012.0,-3098.0,237220.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.88,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.59,4.97,15.05,20.77,36991.0,11644.0,245788.0,-4.14,51.0,X-MC,2.19,62.0,0.31,1.75,12.24,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.31,-38.87,121.17,35.20,95311.0,-50011.0,78659.0,3.10,34.0,X-SC,16.76,92.0,-0.52,0.56,0.89,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-0.16,-19.97,76.81,41.51,161900.0,-52585.0,210780.0,-4.76,40.0,X-MC,7.68,61.0,-0.32,1.50,0.98,XY25,ATH,FMCG
59,RELAXO,1176.00,0.06,-47.88,192.25,52.31,145439.0,-69509.0,75651.0,-44.54,41.0,X-SC,7.30,91.0,-0.48,0.54,1.26,X40N,NTT,FOOTWEAR
3,ACC,3906.00,2.13,-24.82,118.36,64.15,211722.0,-59071.0,178880.0,-55.88,45.0,X-SC,3.56,82.0,-0.28,1.27,2.13,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,0.62,-17.85,89.26,55.47,178391.0,-43430.0,199856.0,-27.18,32.0,X-MC,14.80,64.0,-0.24,1.42,2.40,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.55,-28.50,99.77,42.84,46374.0,-18525.0,46481.0,-53.17,49.0,X-SC,34.54,83.0,-0.40,0.33,5.16,XY24,NTT,MISC
59,RELAXO,1176.00,0.06,-47.88,192.25,52.31,145439.0,-69509.0,75651.0,-44.54,41.0,X-SC,7.30,91.0,-0.48,0.54,1.26,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.31,-38.87,121.17,35.20,95311.0,-50011.0,78659.0,3.10,34.0,X-SC,16.76,92.0,-0.52,0.56,0.89,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.13,-2.47,27.22,24.08,33704.0,-3131.0,123819.0,-10.30,42.0,X-SC,5.86,89.0,-0.09,0.88,16.67,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.62,-17.54,73.81,43.32,99127.0,-28572.0,134300.0,-29.17,34.0,X-SC,5.44,90.0,-0.29,0.95,3.12,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.28,-8.61,32.07,20.71,110862.0,-32551.0,345686.0,-21.36,73.0,X-LC,3.66,1.0,-0.29,2.46,15.51,X40,ATH,IT
41,INFY,1972.00,3.10,15.01,16.72,34.24,60169.0,46976.0,359864.0,-10.27,76.0,X-LC,5.35,2.0,0.78,2.56,24.54,X40,NTT,IT
75,TMPV,600.00,1.86,-26.38,67.04,22.98,111494.0,-59588.0,166310.0,-23.08,50.0,X-LC,6.55,3.0,-0.53,1.18,4.60,XY24,NTT,AUTO
81,VBL,671.64,3.33,-2.02,38.47,35.67,119050.0,-6380.0,309462.0,-13.74,60.0,X-LC,3.77,4.0,-0.05,2.20,11.55,X40N,ATH,FMCG
43,ITC,452.00,0.42,-1.29,12.23,10.78,29012.0,-3098.0,237220.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.88,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.44,-36.74,110.06,32.89,53757.0,-28364.0,48843.0,-703.29,58.0,L-MC,6.62,259.0,-0.53,0.35,34.51,XR,NTT,BANKS
56,RAJESHEXPO,518.00,13.09,-55.35,124.13,0.07,76026.0,-75930.0,61247.0,2144.95,79.0,L-SC,3.14,268.0,-1.00,0.44,52.32,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,2.68,-1.90,85.19,81.67,78401.0,-1779.0,92031.0,8288.89,74.0,L-SC,11.62,271.0,-0.02,0.65,78.22,XR,NTT,CERAMICS
50,MASFIN,398.61,0.08,-3.18,26.06,22.05,24721.0,-3120.0,94860.0,-16.44,51.0,H-SC,6.50,164.0,-0.13,0.67,37.89,XR,ATH,FINANCE
13,BSOFT,831.70,3.71,-18.50,85.01,50.79,97451.0,-26018.0,114635.0,4.37,72.0,H-SC,7.25,151.0,-0.27,0.81,33.47,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,13.09,-55.35,124.13,0.07,76026.0,-75930.0,61247.0,2144.95,79.0,L-SC,3.14,268.0,-1.00,0.44,52.32,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,3.71,-18.50,85.01,50.79,97451.0,-26018.0,114635.0,4.37,72.0,H-SC,7.25,151.0,-0.27,0.81,33.47,XR,ATH,IT
6,ASIANTILES,137.00,2.68,-1.90,85.19,81.67,78401.0,-1779.0,92031.0,8288.89,74.0,L-SC,11.62,271.0,-0.02,0.65,78.22,XR,NTT,CERAMICS
61,ROUTE,2227.21,-0.48,-45.62,210.87,69.05,152588.0,-60705.0,72361.0,-55.87,55.0,H-SC,21.35,140.0,-0.40,0.51,11.60,SR,ATH,IT
84,WIPRO,420.00,3.14,11.87,53.98,72.27,102329.0,20120.0,189568.0,-3.81,77.0,M-LC,3.38,99.0,0.20,1.35,18.78,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.48
1,25,44.88
2,50,76.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.89
MC    30.74
LC    24.37
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.72
X40      23.11
X40N     12.96
XR        9.85
XY25      9.07
AR        8.91
OX40N     7.63
SR        1.00
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.25
H-SC    23.66
X-LC    20.88
M-SC    11.65
X-SC     8.02
H-MC     4.79
L-SC     1.56
M-LC     1.35
M-MC     1.35
H-LC     1.12
L-LC     1.02
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.25,-7.51,42.08
IT,13.22,-13.05,69.89
FINANCE,9.46,-19.55,71.54
MISC,7.27,-27.11,79.11
ELECTRICAL,5.98,-11.68,52.58
PAINTS,5.79,-13.45,30.30
INSURANCE,4.77,-1.22,35.86
PHARMA,4.07,-2.55,34.83
AUTO,2.85,-30.57,75.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3162123.0,21
AR,1318780.0,10
XR,1274049.0,13
X40,972592.0,14
X40N,863966.0,9
OX40N,718659.0,10
XY25,366269.0,6
SR,278885.0,2
MH,78232.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3647156.0,25
M-SC,1419784.0,15
X-MC,1415298.0,16
X-SC,790526.0,8
X-LC,789719.0,11
H-MC,404884.0,3
L-SC,241462.0,3
M-LC,102329.0,1
H-LC,76636.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1275881.0      6
           AR         904980.0      5
M-SC       XY24       810114.0      6
H-SC       XR         779901.0      7
X-MC       X40        483531.0      7
           XY24       405273.0      3
           X40N       343635.0      4
H-SC       OX40N      329277.0      4
M-SC       OX40N      313356.0      5
X-SC       X40N       306846.0      3
X-LC       X40        297181.0      5
X-SC       XY24       291800.0      3
H-SC       SR         278885.0      2
X-LC       X40N       213485.0      2
H-MC       AR         213476.0      2
X-SC       X40        191880.0      2
H-MC       XY24       191408.0      1
X-LC       XY24       187647.0      2
X-MC       XY25       182859.0      2
M-SC       XR         172626.0      2
L-SC       XR         165436.0      2
M-SC       AR         123688.0      2
M-LC       XR         102329.0      1
X-LC       XY25        91406.0      2
H-SC       MH          78232.0      1
H-LC       AR          76636.0      1
L-SC       OX40N       76026.0      1
M-MC       XY25        55600.0      1
L-MC       XR          53757.0      1
L-LC       XY25        36404.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
